In [54]:
import bs4
import requests
import re
import os
from lxml import etree
import csv
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

all_info_list = []    # 所有数据的列表
headers = {'user-agent':'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.74 Safari/537.36'}

def get_info(url):
    session = requests.Session()
    response = session.get(url)
    if not response:
        return

    content_html = response.content.decode('utf-8')
    bs_content = bs4.BeautifulSoup(content_html, "html.parser")
    # 找出所需数据的区域
    results_li = bs_content.find_all(name='li', attrs={'class': re.compile(r'^course')})
    results_div = bs_content.find_all(name='div', attrs={'class': 'item-line item-line--bottom'})
    results = zip(results_li, results_div)
    
    # 解析出想要的数据元素
    for result in results:
        item = {}
        course = result[0].h4.a['title']
        price = result[1].find(name='span', attrs={'class':re.compile(r'item-price')}).text
        users = result[1].find(name='span', attrs={'class':'line-cell item-user custom-string'}).text
        count = result[0].find(name='div', attrs={'class':'item-line item-line--middle'}).span.text
        agency = result[0].find(name='div', attrs={'class':'item-line item-line--middle'}).a['title']
        
        if price == '免费':
            price='0'
            
        price = float(re.findall(r"\d+\.?\d*" ,price)[0])
        users = int(re.findall(r"\d+\.?\d*" ,users)[0])
        
        item = [course, price, users, count, agency]
        all_info_list.append(item)
            

In [39]:
urls = ['https://ke.qq.com/course/list?mt=1001&page={}'.format(i) for i in range(1,34)]
for url in urls:
    get_info(url)
    
# 将爬取的数据写入csv中
with open('Data.csv', 'w', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['课程名称', '价格', '报名人数', '课程进度', '课程机构'])
    for item in all_info_list:
        csv_writer.writerow(item)
    

In [62]:
# 分析数据
fd = open('Data.csv', 'r')
alg = pd.read_csv(fd, header=0)
print(alg.shape)

gr_agency = alg.groupby(['课程机构'])

def func_handle(g, g_list):
    return pd.Series(data=g.count(), index=g_list)

# 统计每个教育机构开设的课程数量
re = gr_agency.apply(func_handle, g_list=['课程名称'])
re = re.rename(columns={'课程名称':'课程数量'})
print(re)


(792, 5)
                课程数量
课程机构                
IT老兵               1
SPOTO思博网络         23
TWeb Conf          1
hcoder 刘海君         1
【智能社】在线课堂          3
...              ...
鲁班学院              22
鸟哥教育               1
黄埔课堂               2
黄菊华                5
鼎智传承YESLAB网络培训     1

[120 rows x 1 columns]
